In [23]:
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os

import urllib.request
import urllib


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.

query_txt = 'Gmarket'
query_url='http://corners.gmarket.co.kr/Bestsellers'

cnt = int(input('  1.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 20)

f_dir=input('  2.파일이 저장될 경로만 쓰세요;(예:c:\\temp\\): ')
print("\n")

if cnt > 30 :
      print("  요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("  요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")
      
    # 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

img_dir = f_dir+s+'-'+query_txt+"\\images"

os.makedirs(img_dir)

os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

path = "c:/Temp/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get(query_url)
time.sleep(1)

def scroll_down(driver):
      driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
      time.sleep(1)


i = 1
while (i <= 3):
         scroll_down(driver) 
         i += 1


rank = []
product = []
o_price = []
s_price = []
per = []
photo2 = []
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
        
count = 1
img_file_no = 0
sale_result = soup.select('div.best-list')
slist = sale_result[1].select('ul > li')

for li in slist:
                
            os.chdir(img_dir)

            try :
                  photo = li.find('div','thumb').find('img')['src']
            except AttributeError :
                continue
            
            img_file_no += 1

            urllib.request.urlretrieve(photo,str(img_file_no)+'.jpg')
            time.sleep(2)

            if img_file_no > cnt :
                break
             
            photo2.append(photo)
            
        
    
for li in slist:
    
    f = open(ff_name, 'a',encoding='UTF-8')
    f.write("-----------------------------------------------------"+"\n")
        
    try:
        rank2 = li.find('p').get_text()

    except AttributeError :
        rank2 = ''
        print('1.판매순위:',rank2.replace("\n",""))
    else:
        print('1. 판매순위 : ',rank2)
        f.write('1.판매순위:'+ rank2 + "\n")
            
            
    try:
        product2 = li.find('a',class_="itemname").get_text()

    except AttributeError :
        product2 = ''
        print('2.제품소개:',product2.replace("\n",""))
        f.write('2.제품소개:'+ product2 + "\n")
    else:
        print('2.제품소개 : ',product2)
        f.write('2.제품소개:'+ product2 + "\n")
            
            
    try:
        o_price2 = li.find('div', class_="o-price").find('span').get_text()
    except AttributeError :
        o_price2 = ''
        print('3.원래가격:',o_price2.replace("\n",""))
        f.write('3.원래가격:'+ o_price2 + "\n")
    else:
        print('3.원래가격 : ',o_price2)
        f.write('13.원래가격:'+ o_price2 + "\n")
            
    try:
        s_price2 = li.find('div', class_="s-price").find('strong').get_text()
    except AttributeError :
        s_price2 = ''
        print('4. 판매가격:',s_price2.replace("\n",""))
        f.write('4. 판매가격:'+ s_price2 + "\n")
    else:
        print('4. 판매가격 : ',s_price2)
        f.write('4. 판매가격:'+ s_price2 + "\n")
                  
            
    try:
        per2 = li.find('div',class_ = "s-price").find('em').get_text()

    except AttributeError :
        rank2 = ''
        print('5. 할인율:',per2.replace("\n",""))
        f.write('5. 할인율:'+ per2 + "\n")
    else:
        print('5. 할인율 : ',per2 + '\n')
        f.write('5. 할인율:'+ per2 + "\n")
        
        
    rank.append(rank2)
    product.append(product2.replace("\n",""))
    o_price.append(o_price2.replace("\n",""))
    s_price.append(s_price2.replace("\n",""))
    per.append(per2.replace("\n",""))
    
    
    if count == cnt :
        break
        
    count = count+1
    time.sleep(1)
    
    
g_best_seller = pd.DataFrame()

g_best_seller['판매순위']=rank
g_best_seller['제품소개']=pd.Series(product)
g_best_seller['원래가격']=pd.Series(o_price)
g_best_seller['판매가격']=pd.Series(s_price)
g_best_seller['할인율']=pd.Series(per)
g_best_seller['사진']=pd.Series(photo2)

g_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

g_best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 건수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

  1.크롤링 할 건수는 몇건입니까?: 10
  2.파일이 저장될 경로만 쓰세요;(예:c:\temp\): c:\


  요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~
1. 판매순위 :  1
2.제품소개 :  [크리넥스]3겹 순수소프트 화장지 27M 30롤X2팩/휴지 /new
3.원래가격 :  42,300원
4. 판매가격 :  31,500원
5. 할인율 :  25%

1. 판매순위 :  2
2.제품소개 :  [바다원]진미채 1kg 국내가공 오징어채
3.원래가격 :  29,100원
4. 판매가격 :  16,000원
5. 할인율 :  45%

1. 판매순위 :  3
2.제품소개 :  청정원 T 청정원 안주야 10종 택4 골라담기 + 증정
3.원래가격 :  19,900원
4. 판매가격 :  17,900원
5. 할인율 :  10%

1. 판매순위 :  4
2.제품소개 :  쇼인더룸 15%쿠폰특가 여름신상 티셔츠/팬츠/빅사이즈
3.원래가격 :  42,900원
4. 판매가격 :  12,900원
5. 할인율 :  69%

1. 판매순위 :  5
2.제품소개 :  옛날과자 전통 고급종합전병 1kg+1kg
3.원래가격 :  17,800원
4. 판매가격 :  8,900원
5. 할인율 :  50%

1. 판매순위 :  6
2.제품소개 :  [프롬산타]3+1 봄 신상 면 덧신/양말/스타킹 남자여자발목학생
3.원래가격 :  23,000원
4. 판매가격 :  6,900원
5. 할인율 :  70%

1. 판매순위 :  7
2.제품소개 :  [니]NII 남녀 여름티셔츠외50종+누구나17%추가할인
3.원래가격 :  33,000원
4. 판매가격 :  9,900원
5. 할인율 :  70%

1. 판매순위 :  8
2.제품소개 :  [크리넥스]크리넥스 실크소프트 250매X3입X3팩 /곽티슈
3.원래가격 :  27,000원
4. 판매가격 :  17,900원
5. 할인율 :  33%

1. 판매순위 :  9
2.제품소개 :  브리치X저스트원 BEST 15%쿠폰 셔츠/팬츠/스커트
3.원래가